In [16]:
import mxnet as mx
import numpy as np
import os, time, shutil

from mxnet import gluon, image, init, nd
from mxnet import autograd as ag
from mxnet.gluon import nn
from mxnet.gluon.model_zoo import vision as models
from mxnet.gluon.data.vision import transforms
from gluoncv.utils import makedirs


!rm -f /dev/shm/mx* 

In [17]:
import os
os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = str(0)

In [18]:
600000/5000.0

120.0

In [19]:
classes = 3

epochs = 350
lr = 0.004
per_device_batch_size = 10
momentum = 0.9
wd = 0.0001

lr_factor = 0.75
lr_steps = [15, 30, 40, 80,160,250,np.inf]

num_gpus = 1
num_workers = 4
# ctx = [mx.gpu(i) for i in range(num_gpus)] if num_gpus > 0 else [mx.cpu()]
# ctx = [mx.cpu()]
batch_size = per_device_batch_size * max(num_gpus, 1)
ctx = [mx.gpu(1)]

In [20]:
ctx

[gpu(1)]

In [ ]:
!nvidia-smi

In [22]:
jitter_param = 0.4
lighting_param = 0.1

transform_train = transforms.Compose([
    transforms.Resize(250),
    transforms.RandomResizedCrop((224,224), scale=(0.8, 1.0), ratio=(0.8, 1.2), interpolation=1),
    transforms.RandomFlipLeftRight(),
    transforms.RandomFlipTopBottom(),
    transforms.RandomBrightness(0.2),
    transforms.RandomLighting(0.2),
    transforms.RandomContrast(0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
 ])


transform_test = transforms.Compose([
    transforms.Resize(250),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [23]:
path = '/xuan/data/AB_PREDICT_RESULT/TESTRESULTFORTRANS/modelData4/'
train_path = os.path.join(path, 'TRAIN')
val_path = os.path.join(path, 'VAL')
test_path = os.path.join(path, 'TEST')

train_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(train_path).transform_first(transform_train),
    batch_size=batch_size, shuffle=True, num_workers=num_workers)

val_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(val_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

test_data = gluon.data.DataLoader(
    gluon.data.vision.ImageFolderDataset(test_path).transform_first(transform_test),
    batch_size=batch_size, shuffle=False, num_workers = num_workers)

In [24]:
model_name = 'resnet50_v2'
finetune_net = gluon.model_zoo.vision.get_model(model_name, pretrained=True)
with finetune_net.name_scope():
    finetune_net.output = nn.Dense(classes)
# finetune_net.features.initialize(init.Xavier(), ctx = ctx)
finetune_net.output.initialize(init.Xavier(), ctx = ctx)
finetune_net.collect_params().reset_ctx(ctx)
finetune_net.hybridize()

trainer = gluon.Trainer(finetune_net.collect_params(), 'sgd', {
                        'learning_rate': lr, 'momentum': momentum, 'wd': wd})
metric = mx.metric.Accuracy()
L = gluon.loss.SoftmaxCrossEntropyLoss()

In [25]:
finetune_net

ResNetV2(
  (features): HybridSequential(
    (0): BatchNorm(fix_gamma=True, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=3)
    (1): Conv2D(3 -> 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (2): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
    (3): Activation(relu)
    (4): MaxPool2D(size=(3, 3), stride=(2, 2), padding=(1, 1), ceil_mode=False)
    (5): HybridSequential(
      (0): BottleneckV2(
        (bn1): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
        (bn3): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
        (bn2): BatchNorm(fix_gamma=False, use_global_stats=False, eps=1e-05, momentum=0.9, axis=1, in_channels=64)
        (conv3): Conv2D(64 -> 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (conv2): Conv2D(64 -> 64, kernel_size=(3, 3), stride=(1, 1), 

In [26]:
def testAll(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        metric.update(label, outputs)
    return metric.get()



def test0(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        labelForTrue = []
        outputsForTrue = []
        for idx in range(len(label)):
            flag = label[idx].asnumpy() == 0
            labelForTrue.append(mx.ndarray.array(label[idx].asnumpy()[flag]))
            outputsForTrue.append(mx.ndarray.array(outputs[idx].asnumpy()[flag]))
        metric.update(labelForTrue, outputsForTrue)
    return metric.get()
def test1(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        labelForTrue = []
        outputsForTrue = []
        for idx in range(len(label)):
            flag = label[idx].asnumpy() == 1
            labelForTrue.append(mx.ndarray.array(label[idx].asnumpy()[flag]))
            outputsForTrue.append(mx.ndarray.array(outputs[idx].asnumpy()[flag]))
        metric.update(labelForTrue, outputsForTrue)
    return metric.get()


def test2(net, val_data, ctx):
    metric = mx.metric.Accuracy()
    for i, batch in enumerate(val_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
        outputs = [net(X) for X in data]
        labelForTrue = []
        outputsForTrue = []
        for idx in range(len(label)):
            flag = label[idx].asnumpy() == 2
            labelForTrue.append(mx.ndarray.array(label[idx].asnumpy()[flag]))
            outputsForTrue.append(mx.ndarray.array(outputs[idx].asnumpy()[flag]))
        metric.update(labelForTrue, outputsForTrue)
    return metric.get()



In [ ]:
lr_counter = 0
num_batch = len(train_data)
resultList = []
for epoch in range(epochs):
    if epoch == lr_steps[lr_counter]:
        trainer.set_learning_rate(trainer.learning_rate*lr_factor)
        lr_counter += 1

    tic = time.time()
    train_loss = 0
    metric.reset()
    for i, batch in enumerate(train_data):
        data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
        label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
#         print(data[0].shape)
        with ag.record():
            outputs = [finetune_net(X) for X in data]
            loss = [L(yhat, y) for yhat, y in zip(outputs, label)]
        for l in loss:
            l.backward()

        trainer.step(batch_size)
        train_loss += sum([l.mean().asscalar() for l in loss]) / len(loss)

        metric.update(label, outputs)
    
    _, train_acc = metric.get()
    train_loss /= num_batch

    _, val_acc_0 = test0(finetune_net, val_data, ctx)
    _, val_acc_1 = test1(finetune_net, val_data, ctx)
    _, val_acc_2 = test2(finetune_net, val_data, ctx)
    _, val_acc_all = testAll(finetune_net, val_data, ctx)
    
    
    print('[Epoch %d] Train-acc: %.3f, loss: %.3f | Val-acc_0: %.3f |Val-acc_1: %.3f |Val-acc_2: %.3f|Val-acc_all: %.3f | time: %.1f' %
             (epoch, train_acc, train_loss, val_acc_0,val_acc_1,val_acc_2,val_acc_all, time.time() - tic))

    resultList.append((epoch, train_acc, train_loss, val_acc_0,val_acc_1,val_acc_2,val_acc_all, time.time() - tic))
    if epoch %5 == 0 :
        file_name = "/xuan/data/AB_PREDICT_RESULT/TESTRESULTFORTRANS/modelData4/model/model_epoch" +str(epoch) +  ".params"
        finetune_net.save_params(file_name)
    
_, test_acc = testAll(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

file_name ="/xuan/data/AB_PREDICT_RESULT/TESTRESULTFORTRANS/modelData4/model/model_final3yinlie.params"
finetune_net.save_params(file_name)


[Epoch 0] Train-acc: 0.716, loss: 0.690 | Val-acc_0: 0.609 |Val-acc_1: 0.649 |Val-acc_2: 0.920|Val-acc_all: 0.784 | time: 56.1
[Epoch 1] Train-acc: 0.785, loss: 0.539 | Val-acc_0: 0.376 |Val-acc_1: 0.807 |Val-acc_2: 0.907|Val-acc_all: 0.789 | time: 55.5
[Epoch 2] Train-acc: 0.797, loss: 0.509 | Val-acc_0: 0.451 |Val-acc_1: 0.879 |Val-acc_2: 0.868|Val-acc_all: 0.804 | time: 57.0
[Epoch 3] Train-acc: 0.823, loss: 0.451 | Val-acc_0: 0.756 |Val-acc_1: 0.766 |Val-acc_2: 0.923|Val-acc_all: 0.847 | time: 56.4
[Epoch 4] Train-acc: 0.830, loss: 0.435 | Val-acc_0: 0.808 |Val-acc_1: 0.721 |Val-acc_2: 0.924|Val-acc_all: 0.842 | time: 56.7
[Epoch 5] Train-acc: 0.832, loss: 0.454 | Val-acc_0: 0.568 |Val-acc_1: 0.901 |Val-acc_2: 0.770|Val-acc_all: 0.778 | time: 56.3
[Epoch 6] Train-acc: 0.842, loss: 0.411 | Val-acc_0: 0.703 |Val-acc_1: 0.850 |Val-acc_2: 0.906|Val-acc_all: 0.855 | time: 57.1


In [ ]:
print(resultList)

In [28]:
_, test_acc = test0(finetune_net, test_data, ctx)
print('[Finished] Test-acc_n: %.3f' % (test_acc))
_, test_acc = test1(finetune_net, test_data, ctx)
print('[Finished] Test-acc_t: %.3f' % (test_acc))
_, test_acc = testAll(finetune_net, test_data, ctx)
print('[Finished] Test-acc: %.3f' % (test_acc))

[Finished] Test-acc_n: 0.752
[Finished] Test-acc_t: 0.871
[Finished] Test-acc: 0.896


# save模型参数

In [14]:
file_name = "/xuan/detection/data/yinlieData/yinlie_vs_true_padding/model/model_final.params"
finetune_net.save_params(file_name)



# load 模型参数


In [21]:

# finetune_net = gluon.model_zoo.vision.get_model(model_name, pretrained=False)
# finetune_net.load_params(file_name,ctx= ctx)



## 测试

In [29]:
from mxnet import nd
from mxnet import gluon
from mxnet.gluon import nn
from mxnet.gluon.data.vision import datasets, transforms
import matplotlib.pyplot as plt

In [34]:


import cv2



transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


finetune_net = gluon.model_zoo.vision.get_model(model_name, pretrained=False)
with finetune_net.name_scope():
    finetune_net.output = nn.Dense(classes)
finetune_net.load_params(file_name,ctx= ctx[0])

import glob
data = [ data for data  in  glob.iglob('/xuan/detection/data/yinlieData/yinlie_vs_true_padding/TEST/very_yinlie/*') ]
for file in data:
    img = cv2.imread(file)
    imgaug = transform_test(mx.ndarray.array(img)).expand_dims(axis=0)
    imgaug = imgaug.as_in_context(mx.gpu(2))
    print(finetune_net(imgaug))

In [32]:
for i, batch in enumerate(val_data):
    data = gluon.utils.split_and_load(batch[0], ctx_list=ctx, batch_axis=0, even_split=False)
    label = gluon.utils.split_and_load(batch[1], ctx_list=ctx, batch_axis=0, even_split=False)
    break

In [33]:
(data[0].shape)

(10L, 3L, 224L, 224L)